In [1]:
%load_ext autoreload
%autoreload 2

import multiprocessing
multiprocessing.set_start_method('fork')

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

import warnings
import numpy as np
import pandas as pd


from functools import partial
from ptls.data_load.datasets import SyntheticDataset, ParquetFiles, ParquetDataset
from ptls.frames.supervised import SeqToTargetDataset, SeqToTargetIterableDataset, SequenceToTarget
from ptls.frames import PtlsDataModule
from functools import partial


from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices

warnings.filterwarnings('ignore')
# logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

In [2]:
from ptls.data_load.datasets import Config, MonoTargetSyntheticDatasetWriter
seq_len = 1024

def get_config(noise):
    '''
    chain_confs = {
        "A": (4, 4, noise),
        "B": (8, 16, 0),
        "C": (4, 4, noise / 4),
        "D": (8, 16, 0)
    }
    '''
    chain_confs = {
        "A": (8, 8, noise),
        "B": (8, 16, noise),
    }

    #state_from = ["A", "C"]
    #state_to = ["B", "D"]
    state_from = []
    state_to = []

    '''
    labeling_conf = {
        0: {"B": 0.8}
    }
    '''
    labeling_conf = {
        0: {"A": 0.8,
            "B": 0.4}
    }

    config = Config(chain_confs, state_from, state_to, labeling_conf)
    return config

config = get_config(0)

In [3]:
writer = MonoTargetSyntheticDatasetWriter(config, path="syndata/test_qwe", seq_len=1024,
                                          n_train_files=250, n_eval_files=50, n_test_files=0,
                                          train_per_file=256*4, eval_per_file=256*4, test_per_file=256*4,
                                          save_config_name="qweqwe", n_procs=16)

In [4]:
writer.write_dataset()

  0%|                                                                                                                                                                                                                 | 0/250 [00:00<?, ?it/s]

0.07954597473144531


  0%|▊                                                                                                                                                                                                        | 1/250 [00:06<27:33,  6.64s/it]

0.08025383949279785


  1%|█▌                                                                                                                                                                                                       | 2/250 [00:13<27:03,  6.55s/it]

...

  8%|███████████████▏                                                                                                                                                                                        | 19/250 [02:11<26:35,  6.91s/it]

0.09537196159362793


  8%|████████████████                                                                                                                                                                                        | 20/250 [02:18<26:35,  6.94s/it]

0.09324383735656738


  8%|████████████████                                                                                                                                                                                        | 20/250 [02:25<27:53,  7.28s/it]

KeyboardInterrupt

